In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [8]:
data_dir = "/Users/torresmateo/OneDrive - FGV/prot2vec"

In [9]:
vecs = pd.read_pickle(os.path.join(data_dir, "[SiameseSimilarityNet-relu]-200_epochs.pt-representations.pkl"))

In [11]:
vecs

,protein,vector,set
0,C1P607,"[0.0, 0.0, 0.028081667, 0.0, 0.0, 0.0, 0.05079...",train
1,C1P615,"[0.0, 0.0, 0.028081667, 0.0, 0.0, 0.0, 0.05079...",train
2,O32528,"[0.0024854182, 0.006133383, 0.055946045, 0.002...",train
3,P00350,"[0.0, 0.0030821683, 0.036483362, 0.0026226195,...",train
4,P00363,"[0.0, 0.006019949, 0.039784305, 0.008943755, 0...",train
...,...,...,...
2267,Q47149,"[0.00083081634, 0.008667405, 0.0, 0.013155985,...",test
2268,Q47538,"[0.0042851022, 0.011407573, 0.0, 0.0080877915,...",test
2269,Q47689,"[0.00094289955, 0.0055822255, 0.011060322, 0.0...",test
2270,Q57261,"[0.00040532384, 0.0039133285, 0.027630303, 0.0...",test
